In [64]:
import tensorflow as tf
# Tensor flow comes with example data
from tensorflow.examples.tutorials.mnist import input_data
import inspect
import numpy as np

In [2]:
type(input_data)

module

In [3]:
dir(input_data)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'division',
 'gzip',
 'numpy',
 'os',
 'print_function',
 'read_data_sets',
 'tempfile',
 'tf',
 'urllib',
 'xrange']

In [25]:
mnist = input_data.read_data_sets("MNIST/data", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST/data/t10k-labels-idx1-ubyte.gz


In [60]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [61]:
dir(mnist)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_make',
 '_replace',
 '_source',
 'count',
 'index',
 'test',
 'train',
 'validation']

In [62]:
type(mnist.train.images)

numpy.ndarray

In [63]:
type(mnist.train.images[0])

numpy.ndarray

In [49]:
len(mnist.train.images)

55000

In [50]:
print(mnist.train.images[0])

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [51]:
print(mnist.train.labels[0])

[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]


In [55]:
print("shape of images = %s" % (np.shape(mnist.train.images),))

shape of lables = (55000, 784)


In [56]:
# Shapes are one hot encoded vector
print("shape of lables = %s" % (np.shape(mnist.train.labels),))

shape of lables = (55000, 10)


In [67]:
# Create a trainable linear model y = Wx + b. Use tf.Variables for weights and bias for training and
# placeholders for training data
# Ensure dimensionality of the matrices match, 'None' is used to specify unknown/variable dimension
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# model
y = tf.matmul(x, W) + b
#lables
y_ = tf.placeholder(dtype=tf.float32, shape=[None, 10])

In [70]:
# We use cross entropy as the loss function we want to minimize. 
# Cross entropy describes the number of questions(information) needed to distinguish 
# one digit from another. It is a function of the probability distribution of each possible 
# outcome. -(Sum[y^log(y)]) where y^ is the true probability (label) and y is the predicted outcome
# The raw formulation of cross-entropy,
  #
  #   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
  #                                 reduction_indices=[1]))
  #
  # can be numerically unstable.
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.

cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y))
#define training step as Gradient descent to minimize the cross entropy
training_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)

In [76]:
# All the training samples can't be used in training.
# Get random batches of 100 samples and train weights and bias
session = tf.InteractiveSession()
tf.global_variables_initializer().run()

for _ in range(10000):
    batch_x, batch_y = mnist.train.next_batch(batch_size=100)
    session.run(training_step, feed_dict={x:batch_x, y_:batch_y})


In [77]:
# test trained model
correct_predictions = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [78]:
print(session.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

0.9154
